In [21]:
!pip install transformers torch numpy pandas scikit-learn matplotlib

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset,random_split
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [23]:
import torch.nn as nn
from transformers import BertModel, BertConfig

class AnglePredictor(nn.Module):
    def __init__(self):
        super(AnglePredictor, self).__init__()
        config = BertConfig(
            hidden_size=128,  
            num_attention_heads=2,  
            num_hidden_layers=2, 
            intermediate_size=128,  
            max_position_embeddings=12246  
        )

        self.embeddings = nn.Linear(12, 128)  
        self.transformer = BertModel(config)
        self.fc = nn.Linear(128, 14)  

    def forward(self, x):
        x = self.embeddings(x)  
        attention_mask = torch.ones(x.size()[:-1], device=x.device)  
        x = self.transformer(inputs_embeds=x, attention_mask=attention_mask)[0]
        x = self.fc(x)
        return x

In [24]:
# Load data
X_train = np.load('/kaggle/input/mydata/X_train_padding.npy')
y_train = np.load('/kaggle/input/mydata/y_train_padding.npy')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [25]:
# Initialize the model
model = AnglePredictor()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [26]:
num_epochs = 10
train_losses = []
test_losses = []

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)  # Batch size of 1 due to sequence length

# Training loop
epochs = 20
model.train()
for epoch in range(epochs):
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        print(f'Epoch {epoch}/{epochs}, Loss: {epoch_loss/len(train_loader)}')
    print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader)}')

Epoch 0/20, Loss: 8.243240921585649
Epoch 0/20, Loss: 26.15350115740741
Epoch 0/20, Loss: 36.12481840157214
Epoch 0/20, Loss: 46.4441362545814
Epoch 0/20, Loss: 52.8715503833912
Epoch 0/20, Loss: 65.29637843002507
Epoch 0/20, Loss: 84.97649694372106
Epoch 0/20, Loss: 102.15033863208912
Epoch 0/20, Loss: 112.7909696602527
Epoch 0/20, Loss: 124.39739161361882
Epoch 0/20, Loss: 138.38001693913967
Epoch 0/20, Loss: 145.40054811077354
Epoch 0/20, Loss: 159.68886763961225
Epoch 0/20, Loss: 169.14690709997106
Epoch 0/20, Loss: 177.4495691370081
Epoch 0/20, Loss: 183.31481707537617
Epoch 0/20, Loss: 193.46425148292823
Epoch 0/20, Loss: 204.47840711805554
Epoch 0/20, Loss: 215.29619532455632
Epoch 0/20, Loss: 224.06606415171683
Epoch 0/20, Loss: 230.89666521990742
Epoch 0/20, Loss: 240.88370542173033
Epoch 0/20, Loss: 250.7502803096065
Epoch 0/20, Loss: 261.7063372576678
Epoch 0/20, Loss: 267.17965924298323
Epoch 0/20, Loss: 274.91635357892073
Epoch 0/20, Loss: 283.88735057689524
Epoch 0/20, Lo

In [ ]:
# Evaluating the model
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)
    print(f'Test Loss: {test_loss.item()}')

# Visualize some predictions
import matplotlib.pyplot as plt

plt.plot(y_test_tensor.numpy()[0][:, 0], label='True Angles')
plt.plot(predictions.numpy()[0][:, 0], label='Predicted Angles')
plt.legend()
plt.show()